In [36]:
from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from typing import List, Optional, Dict, Any
import requests
import numpy as np
import faiss
import os
from dotenv import load_dotenv

In [37]:
load_dotenv()

EURI_API_KEY = os.getenv("EURI_API_KEY")

In [38]:
import requests
import numpy as np

def generate_embeddings(text: str):
    url = "https://api.euron.one/api/v1/euri/alpha/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {EURI_API_KEY}"
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    # Convert to numpy array for vector operations
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

In [51]:
from euriai import EuriaiClient

def generate_response(prompt):
    client = EuriaiClient(
        api_key=f"{EURI_API_KEY}",
        model="gpt-4.1-nano"  
    )

    response = client.generate_completion(
        prompt=prompt,
        temperature=0,
        max_tokens=300
    )

    print(response)
    return response['choices'][0]["message"]["content"]

In [40]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

document = TextLoader("sudhanshuinfo.txt").load()
document

[Document(metadata={'source': 'sudhanshuinfo.txt'}, page_content="Making an Impact\nHelping Millions of Students Succeed\nSudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.\n\nIn 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.\n\nThe Entrepreneur and Teacher: Sudhanshu's Dual Legacy\nSudhanshu's journey isn't just one of entrepreneurial success; it's also a story o

In [41]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.split_documents(document)

In [42]:
len(docs)

58

In [43]:
docs

[Document(metadata={'source': 'sudhanshuinfo.txt'}, page_content='Making an Impact\nHelping Millions of Students Succeed'),
 Document(metadata={'source': 'sudhanshuinfo.txt'}, page_content="Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,"),
 Document(metadata={'source': 'sudhanshuinfo.txt'}, page_content='Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.'),
 Document(metadata={'source': 'sudhanshuinfo.txt'}, page_content='In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron co

In [44]:
texts = [doc.page_content for doc in docs]
texts

['Making an Impact\nHelping Millions of Students Succeed',
 "Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,",
 'Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.',
 'In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.',
 "The Entrepreneur and Teacher: Sudhanshu's Dual Legacy",
 "Sudhanshu's journey isn't just one of entrepreneurial success; it's also a story of dedication to teaching. Th

In [13]:
embeddings = [generate_embeddings(i) for i in texts]

KeyError: 'data'

In [45]:
embeddings  = []
valid_texts = []
for i,text in enumerate(texts[0:8]):
    clean_text = text.strip()
    if not clean_text:
        print(f"Skipping empty text at index {i}")
        continue
    embedding = generate_embeddings(text)
    embeddings.append(embedding)
    valid_texts.append(text)
    print(f"Processed {i+1}/{len(texts)}: {text[:30]}... -> {embedding[:5]}")  # Print first 30 characters of text and first 5 elements of embedding

Processed 1/58: Making an Impact
Helping Milli... -> [ 0.03395454 -0.0469909   0.03878181  0.07887272  0.02695909]
Processed 2/58: Sudhanshu's commitment to affo... -> [-0.01364236  0.00639423  0.05721121  0.02440282  0.04653747]
Processed 3/58: Many of these students, like S... -> [-0.02204193 -0.00722562  0.00345266  0.03793968  0.04818918]
Processed 4/58: In 2022, iNeuron was acquired ... -> [-0.04330519 -0.05195538  0.02764536  0.0217475   0.03644469]
Processed 5/58: The Entrepreneur and Teacher: ... -> [0.01671011 0.01011956 0.04287479 0.01667061 0.00875009]
Processed 6/58: Sudhanshu's journey isn't just... -> [-0.00970297 -0.00214104  0.0598214   0.02851626  0.0276796 ]
Processed 7/58: in Big Data, Data Science, or ... -> [-0.00568316 -0.01421538  0.03512035 -0.01586388  0.05972848]
Processed 8/58: His commitment to affordable e... -> [ 0.01117547 -0.04308555  0.04391897  0.05712749  0.01579719]


In [46]:
embeddings

[array([ 0.03395454, -0.0469909 ,  0.03878181, ..., -0.02398636,
        -0.01851818,  0.01633636]),
 array([-0.01364236,  0.00639423,  0.05721121, ..., -0.00413274,
        -0.00975312, -0.00021493]),
 array([-0.02204193, -0.00722562,  0.00345266, ..., -0.00756314,
        -0.02643654, -0.00205438]),
 array([-0.04330519, -0.05195538,  0.02764536, ..., -0.03267549,
         0.01820879,  0.03354321]),
 array([ 0.01671011,  0.01011956,  0.04287479, ..., -0.04371754,
         0.01135734, -0.00684733]),
 array([-0.00970297, -0.00214104,  0.0598214 , ..., -0.01701214,
        -0.01090567,  0.00101315]),
 array([-0.00568316, -0.01421538,  0.03512035, ..., -0.00705393,
         0.00747801,  0.00206959]),
 array([ 0.01117547, -0.04308555,  0.04391897, ...,  0.00480482,
        -0.02457341,  0.01011475])]

In [17]:
import time
import numpy as np

embeddings = []

for i, text in enumerate(texts):
    try:
        print(f"🔄 Processing {i+1}/{len(texts)}...")
        embedding = generate_embeddings(text).astype('float32')
        embeddings.append(embedding)
        time.sleep(1)  # ⏳ Add delay to respect API rate limits
    except ValueError as e:
        print(f"❌ Error on text {i+1}: {e}")


🔄 Processing 1/58...
🔄 Processing 2/58...
🔄 Processing 3/58...
🔄 Processing 4/58...
🔄 Processing 5/58...
🔄 Processing 6/58...
🔄 Processing 7/58...
🔄 Processing 8/58...
🔄 Processing 9/58...
🔄 Processing 10/58...
🔄 Processing 11/58...


KeyError: 'data'

In [31]:
embeddings

[array([ 0.03395454, -0.0469909 ,  0.03878181, ..., -0.02398636,
        -0.01851818,  0.01633636]),
 array([-0.01364236,  0.00639423,  0.05721121, ..., -0.00413274,
        -0.00975312, -0.00021493]),
 array([-0.02204193, -0.00722562,  0.00345266, ..., -0.00756314,
        -0.02643654, -0.00205438]),
 array([-0.04330519, -0.05195538,  0.02764536, ..., -0.03267549,
         0.01820879,  0.03354321]),
 array([ 0.01671011,  0.01011956,  0.04287479, ..., -0.04371754,
         0.01135734, -0.00684733]),
 array([-0.00970297, -0.00214104,  0.0598214 , ..., -0.01701214,
        -0.01090567,  0.00101315]),
 array([-0.00568316, -0.01421538,  0.03512035, ..., -0.00705393,
         0.00747801,  0.00206959]),
 array([ 0.01117547, -0.04308555,  0.04391897, ...,  0.00480482,
        -0.02457341,  0.01011475])]

In [47]:
import faiss
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

In [48]:
dimension = embeddings[0].shape[0]
faiss_index = faiss.IndexFlatL2(dimension)

faiss_index.add(np.array(embeddings).astype('float32'))

In [49]:
faiss_index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x10795b510> >

In [52]:
generate_response("tell me about sudhanshu")

{'id': 'chatcmpl-BW36TRsD3yy8p36aFDCr8ArGP6xH6', 'object': 'chat.completion', 'created': 1746978126, 'model': 'gpt-4.1-nano', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Sure! Could you please provide more context or specify which Sudhanshu you're referring to? There may be multiple individuals with that name. Are you asking about a public figure, a character, or someone else?"}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 25, 'completion_tokens': 44, 'total_tokens': 69}}


"Sure! Could you please provide more context or specify which Sudhanshu you're referring to? There may be multiple individuals with that name. Are you asking about a public figure, a character, or someone else?"

In [53]:
from langchain.chains import RetrievalQA

In [54]:
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

document = [Document(page_content=text) for text in texts[:len(embeddings)]]

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

faiss_index = faiss.IndexFlatL2(dimension)
vector_store = FAISS(
    embedding_function=generate_embeddings,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
vector_store.add_documents(document)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['ca448d9c-4e8d-4e31-a062-ac4ff98c1d97',
 '09894e89-434b-4848-9302-c8f46bad2c33',
 'dd49ad27-2fe0-467b-a5a3-999be127e20d',
 '11598ae7-a5cb-436e-aad0-301edbd3ccfb',
 'd1274efe-579f-49b0-9de9-eb372c69bcf3',
 'ddc7e11c-0fe8-411c-99fb-08b1b953fe3e',
 '66ec682c-1d9d-47c3-a586-a24089da40cc',
 '50d348b1-58b1-4513-931b-140f039838f6']

In [55]:
vector_store.save_local("faiss_index")

In [56]:
retriver = vector_store.as_retriever(search_kwargs={"k": 2})

In [57]:
from euriai import EuriaiLangChainLLM

llm = EuriaiLangChainLLM(
    api_key=f"{EURI_API_KEY}",
    model="gpt-4.1-nano",
    temperature=0,
    max_tokens=300
)

print(llm.invoke("Write a poem about time travel."))

Through veils of years, we softly glide,  
On threads of moments, far and wide,  
A whisper in the shifting breeze,  
A dance with echoes of histories.

We chase the past’s elusive glow,  
Where ancient stories lie below,  
And leap ahead to futures bright,  
Unfolding in the dawn’s first light.

Time’s river flows both swift and slow,  
A winding stream we long to know,  
With every tick, a door swings wide—  
To realms where dreams and destinies collide.

Yet in this voyage, wonder gleams,  
Between the now and what once seemed,  
A timeless quest, both wild and free—  
To glimpse eternity’s mystery.


In [58]:
rag_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriver
    )

In [59]:
rag_chain("who is sudhanshu")

/var/folders/1q/1dwxrmgj67g2z1tccqkb9mg80000gn/T/ipykernel_3411/2566710363.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_chain("who is sudhanshu")


{'query': 'who is sudhanshu',
 'result': "Sudhanshu is an individual who is dedicated to providing affordable education and has made a significant impact on many students' lives by helping them improve their skills, secure jobs, and overcome limitations. He is also a passionate educator with expertise in fields like Big Data and Data Science, and his efforts are recognized and respected by those he has helped."}

In [ ]:
generate_response("who is sudhanshu")
# quering directly from the LLM, doesn't use the vector store so it doesn't have the context of the document.

{'id': 'chatcmpl-BW3SJ0jRAPc31saINXMgaliuNKPh9', 'object': 'chat.completion', 'created': 1746979480, 'model': 'gpt-4.1-nano', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '"Sudhanshu" is a common Indian given name, and without additional context, it\'s difficult to identify a specific individual. There are many people named Sudhanshu, including professionals, students, artists, and others. Could you please provide more details or specify the context in which you\'re referring to Sudhanshu?'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 24, 'completion_tokens': 67, 'total_tokens': 91}}


'"Sudhanshu" is a common Indian given name, and without additional context, it\'s difficult to identify a specific individual. There are many people named Sudhanshu, including professionals, students, artists, and others. Could you please provide more details or specify the context in which you\'re referring to Sudhanshu?'

In [61]:
rag_chain("give me the comapny whre sudhanshu is working")

{'query': 'give me the comapny whre sudhanshu is working',
 'result': "I don't know the name of the company where Sudhanshu is working based on the provided information."}